In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%cd /kaggle/working
!git clone https://github.com/valkryhx/chatGLM-6B-QLoRA

/kaggle/working
fatal: destination path 'chatGLM-6B-QLoRA' already exists and is not an empty directory.


In [3]:
%cd chatGLM-6B-QLoRA/
!git pull --all --force 
!pip install -r requirements.txt

/kaggle/working/chatGLM-6B-QLoRA
Fetching origin
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 30 (delta 19), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (30/30), 18.16 KiB | 170.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   c5fc957..c2a7ed2  main       -> origin/main
Updating c5fc957..c2a7ed2
Fast-forward
 sft_multi_turn_qlora_chatglm2.py                   |   2 +-
 test-new-sft-langchain.ipynb                       |   2 +-
 train_lora_baichuan7b.py                           | 498 +++++++++++++++++++++
 ...50\200\203\344\270\216\347\273\217\351\252\214" |  82 ++++
 4 files changed, 582 insertions(+), 2 deletions(-)
 create mode 100644 train_lora_baichuan7b.py
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 29.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━

In [ ]:
ls

# 测试baichuan2-7b-chat的sft

In [10]:
!git pull --all --force
!deepspeed --include localhost:0,1  train_lora_baichuan7b.py \
  --train_args_json luzi.json \
  --model_name_or_path  baichuan-inc/Baichuan2-7B-Chat \
  --output_dir sft_baichuan2_7b_1010 \
  --num_eval_samples 1 \
  --train_data_path data/alpaca_yunguan_0814 \
  --eval_data_path  data/alpaca_yunguan_0814 \
  --max_length 512 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 1 \
  --learning_rate  1e-6 \
  --num_train_epochs  70  \
  --save_total_limit 2 \
  --use_qlora True \
--deepspeed ds_zero2_config.json

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 796 bytes | 199.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   81622f1..fbd78f5  main       -> origin/main
Updating 81622f1..fbd78f5
Fast-forward
 train_lora_baichuan7b.py | 4 +++-
 1 file changed, 3 insertions(+), 1 deletion(-)
[2023-10-11 15:24:50,145] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[2023-10-11 15:24:55,855] [WARNING] [runner.py:196:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources on

# 测试  新的 train_lora.py

In [ ]:
!git pull --all --force
!deepspeed --include localhost:1  train_lora.py \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b  \
  --output_dir output0908_v2 \
  --resume_from_checkpoint /kaggle/working/chatGLM-6B-QLoRA/output0908/checkpoint-200 \
  --num_eval_samples 1 \
  --train_data_path data/alpaca_yunguan_0814 \
  --eval_data_path  data/alpaca_yunguan_0814 \
  --max_length 1000 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 4 \
  --learning_rate  1e-6 \
  --num_train_epochs  70  \
  --save_total_limit 2 \
  --use_qlora True \
--deepspeed ds_zero2_config.json

# 测试  新的 sft_lora_multi_turn.py

In [ ]:
!git pull --all --force
!deepspeed --include localhost:0,1  sft_lora_multi_turn.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-sharegpt-2k-sft-0908 \
  --num_train_samples -1 \
  --num_eval_samples 2 \
  --train_data_path ./data/sharegpt_data  \
  --eval_data_path  ./data/sharegpt_data    \
  --data_type sharegpt  \
  --max_length 1000 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2  \
  --gradient_accumulation_steps 2 \
  --learning_rate  1.8e-5 \
  --num_train_epochs  40  \
  --save_total_limit 2 \
  --use_qlora True \
--deepspeed ds_zero2_config.json

# 构造和测试langchain prompt 数据 已经将构造的数据上传到github项目

In [ ]:
import json 
import os

template = """现提供如下信息：
history={history}
context={context}
请使用上述信息回答：{question}
"""

parent_dir = "/kaggle/working/chatGLM-6B-QLoRA/data/alpaca_yunguan_0814"
paths = ['yunguan_aug_part_3.json',
         'yunguan_aug_part_2.json',
        "yunguan_aug_part_1.json",
        "refuse_phone.json"]
list_all =[]
for name in paths :
    with open(os.path.join(parent_dir , name) ,"r",encoding="utf-8") as fr:
        list_all.extend([json.loads(item) for item in  fr.readlines() ])

print(len(list_all))


def get_q_a(idx):
    q = list_all[idx]["instruction"]
    a = list_all[idx]["output"]
    return "\n\n问：{q}\n\n答：{a}".format(q=q,a=a)

import random
res = []
idx_list_all = range(len(list_all))
for chosen_i in range(len(list_all)) :
    his_idx = random.choices(idx_list_all,k=2)
    #print(his_idx)
    context_idx = random.choices(idx_list_all,k=2) 
    #print(context_idx)
    context_idx.extend([chosen_i]) # 两个不相关检索答案+ 正确检索答案 ,比如 [1,2,100] 其中100是正确的q和a 等会idx=100的q作为question
    random.shuffle(context_idx) # 打乱上下文的顺序 [1,100,2]
    history = "".join([get_q_a(idx) for idx in his_idx])
    context = "".join([get_q_a(idx) for idx in context_idx])
    question = list_all[chosen_i]["instruction"]
    answer = list_all[chosen_i]["output"]
    res.append({"instruction":template.format(history=history,context=context,question=question)  ,"input":"" , "output":answer })

# print(res[-10]['instruction'])
# print(res[-10]['output'])

max_q ,max_a, max_total = 0,0,0

for item in res:
    max_q = max(max_q, len(item["instruction"]))
    max_a = max(max_a, len(item['output']))
    max_total = max(max_total , len(item["instruction"]) + len(item["output"]) )

with open(f"alpaca_langchain_prompt_yunguan_0922_max_q_{max_q}_max_a_{max_a}_max_total_{max_total}.json","w",encoding="utf-8") as fw:
    for item in res :
        fw.write(json.dumps(item,ensure_ascii=False)+"\n")
print(f"alpaca_langchain_prompt_yunguan_0922_max_q_{max_q}_max_a_{max_a}_max_total_{max_total}.json   done")

In [ ]:

#cat alpaca_langchain_prompt_yunguan_0919.json

# 用alpaca格式的langchain prompt新数据跑

In [ ]:
!git pull --all --force

In [ ]:
!git pull --all --force
!deepspeed --include localhost:0,1  train_lora.py \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b  \
  --output_dir output_langchain_yunguan_0922 \
  --num_eval_samples 100 \
  --train_data_path data/alpaca_langchain_yunguan \
  --eval_data_path  data/alpaca_langchain_yunguan \
  --max_length 1200 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 4 \
  --learning_rate  1e-6 \
  --num_train_epochs  70  \
  --save_total_limit 2 \
  --use_qlora True \
--deepspeed ds_zero2_config.json

# qlora后测试langchain 格式的问答

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig


# 有效peft_model_path = '/kaggle/working/chatGLM-6B-QLoRA/output-sft-qlora-ds-yunguan0712-aug-v3'

peft_model_path= "/kaggle/working/chatGLM-6B-QLoRA/output_langchain_yunguan_0922/checkpoint-200"
config = PeftConfig.from_pretrained(peft_model_path)
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=torch.float16)
# base_model加载时保持和train时完全一致的参数配置
base_model = AutoModel.from_pretrained(config.base_model_name_or_path,
                                       quantization_config=q_config,
                                       trust_remote_code=True,
                                       device_map='auto')

# input_text = '请你作为烽火SSE智能助手，帮我回答下面的问题。新型全电发票目前系统无法识别，要如何录入？'
# print(f'输入：\n{input_text}')
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

# response, history = base_model.chat(tokenizer=tokenizer,history=[], query=input_text)
# print(f'微调前：\n{response}')

# model = PeftModel.from_pretrained(base_model, peft_model_path)
# response, history = model.chat(tokenizer=tokenizer, history=[] ,query=input_text)
# print(f'微调后: \n{response}')
model = PeftModel.from_pretrained(base_model, peft_model_path)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
print("adapter loaded!")


In [ ]:
line = input()
while line:
    if line.strip()=="#QUIT": # 输入#QUIT退出对话循环
        break
    #TEMPLATE = "作为烽火SSE助手，请帮忙回答如下问题：" #计算两个矩阵的乘积："
    #line = PROMPT_TEMPLATE.format_map({'instruction': line})
    if line.strip() in ["我有一个新问题" ,"#CLEAR"]:
        history=[]
        line = input().replace("\\n","\n")
    
    #print(f"【formatted line】={line}")
    response, history = model.chat(tokenizer, line, history=[],top_p= 0.95,
                                  temperature=0.1,num_beams=7,max_length=1024,repetition_penalty=1.1,do_sample=False) # 量化后 num_beams只能=1 否则报错
    print("ChatGLM2-6B：\n", response.replace("\\n",'\n'))
    print("\n------------------------------------------------\n用户：")
    line = input()
    
    print("\n\n")
    

# <font color=red>根据上面的测试 ，很关键的一个参数是num_beams，目前设置 num_beams=7 ,这个值越大，可能的答案路径就越多，最终会从多个答案路径中选择总和概率最大的，测试发现num_beams越大，model表现的越能利用提供的历史对话和检索信息，根据这些信息回答问题的能力就越强。</font>

# 使用不sft的基座模型测试下  感觉不开启do_sample=True 效果就很波动？？？有时候表现可以 有时候又不行
# 并且 基座模型无法开启num_beams =2,3,4 ，只要不是1 运行就报错

In [ ]:


import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig


# 有效peft_model_path = '/kaggle/working/chatGLM-6B-QLoRA/output-sft-qlora-ds-yunguan0712-aug-v3'

#peft_model_path= "/kaggle/working/chatGLM-6B-QLoRA/output_langchain_yunguan_0922/checkpoint-200"
config = PeftConfig.from_pretrained(peft_model_path)
#q_config = BitsAndBytesConfig(load_in_4bit=True,
#                               bnb_4bit_quant_type='nf4',
#                               bnb_4bit_use_double_quant=True,
#                               bnb_4bit_compute_dtype=torch.float16)
# base_model加载时保持和train时完全一致的参数配置
model = AutoModel.from_pretrained(config.base_model_name_or_path,
                                       #quantization_config=q_config,
                                       trust_remote_code=True,
                                       device_map='auto')

# input_text = '请你作为烽火SSE智能助手，帮我回答下面的问题。新型全电发票目前系统无法识别，要如何录入？'
# print(f'输入：\n{input_text}')
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

# response, history = base_model.chat(tokenizer=tokenizer,history=[], query=input_text)
# print(f'微调前：\n{response}')

# model = PeftModel.from_pretrained(base_model, peft_model_path)
# response, history = model.chat(tokenizer=tokenizer, history=[] ,query=input_text)
# print(f'微调后: \n{response}')
#model = PeftModel.from_pretrained(base_model, peft_model_path)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
print("adapter loaded!")

In [ ]:
line = input()
while line:
    if line.strip()=="#QUIT": # 输入#QUIT退出对话循环
        break
    #TEMPLATE = "作为烽火SSE助手，请帮忙回答如下问题：" #计算两个矩阵的乘积："
    #line = PROMPT_TEMPLATE.format_map({'instruction': line})
    if line.strip() in ["我有一个新问题" ,"#CLEAR"]:
        history=[]
        line = input().replace("\\n","\n")
    
    #print(f"【formatted line】={line}")
    response, history = model.chat(tokenizer, line, history=[],top_p= 0.95,
                                  temperature=0.05,num_beams=1,max_length=8192,repetition_penalty=1.1,do_sample=True) # 量化后 num_beams只能=1 否则报错
    print("ChatGLM2-6B：\n", response.replace("\\n",'\n'))
    print("\n------------------------------------------------\n用户：")
    line = input()
    
    print("\n\n")

# 下载

In [ ]:
cp -r /kaggle/working/chatGLM-6B-QLoRA/output_langchain_yunguan_0922/checkpoint-200  /kaggle/working

In [ ]:
  
import os
os.chdir(r'/kaggle/working')
!tar -cvf   langchain_ckpt_200.tar.gz  /kaggle/working/checkpoint-200
#!tar -czf Landscapes.tar.gz images_out/Landscapes
#!tar -czf    basemodel_chatglm2_6b_ckt_0715.tar.gz   /kaggle/working/basemodel_chatglm2_6b_ckt_0715
from IPython.display import FileLink
FileLink(r'langchain_ckpt_200.tar.gz')

In [ ]:
ls -lrth

In [ ]:
ls

In [ ]:
import os
os.chdir(r'/kaggle/working')

#!tar -czf Landscapes.tar.gz images_out/Landscapes
#!tar -czf    basemodel_chatglm2_6b_ckt_0715.tar.gz   /kaggle/working/basemodel_chatglm2_6b_ckt_0715
from IPython.display import FileLink
FileLink(r'langchain_ckpt_200.tar.gz')

In [ ]:
!git pull --all --force
!deepspeed --include localhost:0,1  train_lora.py \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b  \
  --output_dir output_langchain_yunguan_0922_v2 \
  --resume_from_checkpoint /kaggle/working/chatGLM-6B-QLoRA/output_langchain_yunguan_0922/checkpoint-200 \
  --num_eval_samples 100 \
  --train_data_path data/alpaca_langchain_yunguan \
  --eval_data_path  data/alpaca_langchain_yunguan \
  --max_input_length 900 \
  --max_output_length 256 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 4 \
  --learning_rate  1e-6 \
  --num_train_epochs  70  \
  --save_total_limit 2 \
  --use_qlora True \
--deepspeed ds_zero2_config.json